In [1]:
import json
from fuzzywuzzy import fuzz
import psycopg2
import pyperclip
import tkinter as tk
from tkinter import messagebox
import sys


In [2]:
# Чтение json конфига
with open(r'D:\py\av_parcer\config.json', encoding='UTF-8') as file:
    config = json.load(file)

mail_login = config['sender login']
mail_password = config['sender password']
pgre_login = config['postgre login']
pgre_password = config['postgre password']
pgre_host = config['postgre host']
pgre_port = config['postgre port']
pgre_db = config['postgre database']
recipients = config['mail recipients']
poe_api_keys = config['api_keys']
av_key = config['av_x-api-key']

In [3]:
#Подключение к postgres
conn = psycopg2.connect(
    host = pgre_host,
    port = pgre_port,
    database = pgre_db,
    user = pgre_login,
    password = pgre_password
)

In [4]:
parsecursor = conn.cursor()

In [34]:
id = 128412489

In [35]:
known_duplicates_query = """WITH CTE AS (SELECT %s AS search_id)
Select id, date, price
from av_full
where id = (select search_id from CTE)
or duplicate_id = (select search_id from CTE)
order by date asc"""

In [36]:
parsecursor.execute(known_duplicates_query, (id,))

In [37]:
results = parsecursor.fetchall()

In [43]:
if results:
    first = results[0]      # самая ранняя запись (минимальная дата)
    last = results[-1]      # самая поздняя запись (максимальная дата)

    earliest_date = first[1]
    latest_date = last[1]
    date_range = latest_date - earliest_date  # timedelta

    first_price = first[2]
    last_price = last[2]
    price_diff = last_price - first_price

    max_price = max(row[2] for row in results)  # максимальная цена в выборке
else:
    # Обработка пустого результата
    earliest_date = latest_date = date_range = price_diff = max_price = None

print(f'''
{first}
{last}
{earliest_date}
{latest_date}
{date_range}
{first_price}
{last_price}
{price_diff}
{max_price}
''')
print(f'В продаже с {earliest_date.day}.{earliest_date.month}.{earliest_date.year} ({date_range.days} дней), цена изменилась на {price_diff} USD')


(Decimal('102376139'), datetime.datetime(2022, 10, 20, 13, 17, 22), Decimal('14000'))
(Decimal('128412489'), datetime.datetime(2026, 2, 12, 10, 10, 5), Decimal('11000'))
2022-10-20 13:17:22
2026-02-12 10:10:05
1210 days, 20:52:43
14000
11000
-3000
14000

В продаже с 20.10.2022 (1210 дней), цена изменилась на -3000 USD


In [5]:
# Функция получения статистики по дубликатам
def return_duplicate_stats(id_to_check, dupl_cursor):
    """По передаче ID возвращает словарь duplicate_stats с информацией о дубликатах для ID"""
    # кверя
    known_duplicates_query = """
    WITH CTE AS (SELECT %s AS search_id)
    Select id, date, price
    from av_full
    where id = (select search_id from CTE)
    or duplicate_id = (select search_id from CTE)
    order by date asc
    """ % id_to_check

    dupl_cursor.execute(known_duplicates_query)

    results = dupl_cursor.fetchall()
    
    # обработка results
    if results:
        first = results[0]      # самая ранняя запись (минимальная дата)
        last = results[-1]      # самая поздняя запись (максимальная дата)

        earliest_date = first[1]
        latest_date = last[1]
        date_range = latest_date - earliest_date  # timedelta

        first_price = first[2]
        last_price = last[2]
        price_diff = last_price - first_price

        max_price = max(row[2] for row in results)  # максимальная цена в выборке
    else:
        # Обработка пустого результата
        earliest_date = latest_date = date_range = price_diff = max_price = None
#'earliest_date_string': f'{earliest_date.day}.{earliest_date.month}.{earliest_date.year} ({date_range.days}',
    # возврат данных
    return {
        'results': results,
        'dupl_count': len(results),
        'earliest_date_string': f'{earliest_date.day}.{earliest_date.month}.{earliest_date.year}',
        'date_diff': date_range.days,
        'price_diff': price_diff
    }
    
    messagebox.showwarning(f"{len(results)} дубликатов у {id_to_check}", f'В продаже с {earliest_date.day}.{earliest_date.month}.{earliest_date.year} ({date_range.days} дней), цена изменилась на {price_diff} USD')

In [6]:
root = tk.Tk()

In [7]:
id_to_check = 128412489

In [8]:
# Окно вывода статистики
dupl_stats_window = tk.Toplevel(root)
dupl_stats_window.title("Найдено Х дубликатов")
dupl_stats_window.attributes('-topmost', True)
dupl_stats_window.geometry('%dx%d+%d+%d' % (400, 75, 1300, 600)) # размер ш.в. и положение (отступы) ш.в.

# дернуть базу
results = return_duplicate_stats(id_to_check, parsecursor)

# Поле текста
formatted_text_string = f"""
Найдено {results['dupl_count']} дубликатов. Цена изменилась на {results['price_diff']}USD. 
Самая ранняя дата - {results['earliest_date_string']} (в продаже {results['date_diff']} дней)'
"""
dupl_stats_text = tk.Label(dupl_stats_window, text=formatted_text_string)
dupl_stats_text.pack(pady=5)

# Поле таблицы



dupl_stats_window.mainloop()

In [8]:
id_to_check = 128412489
return_duplicate_stats(id_to_check)